# Pytanie:
Rozważamy reżyserów urodzonych po roku $Y$. Czy wśród tych reżyserów lepsi są ci, którzy są również aktorami? Bierzemy pod uwagę wyłącznie filmy mające więcej niż $K$ głosów w bazie.

In [1]:
from load_data import *

In [84]:
Y, K, n_films = 1970, 5000, 4

#############################################
path_to_file = "data/name.basics.tsv.gz"
df1 = load_to_dataframe(path_to_file)

df1.loc[:, 'isDirector'] = df1["primaryProfession"].astype(str).str.contains("director")
df1.loc[:, 'isActor'] = df1["primaryProfession"].astype(str).str.contains("actor") | df1["primaryProfession"].astype(str).str.contains("actress")
df1 = df1.loc[df1["isDirector"] == True].drop(columns=["primaryProfession", "isDirector"])
df1 = df1.loc[(df1["birthYear"].notna()) & (df1["knownForTitles"].notna())]
df1 = df1.loc[(df1["birthYear"].astype(int) > Y) & (df1["deathYear"].isna())]
df1 = df1.loc[df1["knownForTitles"].apply(lambda x: len(x.split(",")) >= n_films)]

############################################

path_to_file = "data/title.ratings.tsv.gz"
df2 = load_to_dataframe(path_to_file)

path_to_file = "data/title.basics.tsv.gz"
df3 = load_to_dataframe(path_to_file)

df3 = df3.loc[df3["startYear"].notna()]
df3 = df3.loc[(df3["titleType"] == 'movie') & (df3["startYear"].astype(int) > Y), ["tconst"]]
df2 = df2.merge(df3,
                on="tconst",
                how="right")
df2 = df2.dropna().loc[df2["numVotes"] > K]

############################################

dict_df = {}
for num, i in enumerate(df1.index):
    if num % 500 == 0:
        print(f"Za nami {(num/df1.shape[0])*100:.2f}%")
    knownFor = df1.loc[i, :].knownForTitles.split(",")
    stats = dict(df2.loc[df2["tconst"].isin(knownFor)].loc[:, ["averageRating", "numVotes"]].describe().loc[["mean", "std"], :])
    dict_df[(df1.loc[i, "nconst"], "averageRating")] = dict(stats['averageRating'])
    dict_df[(df1.loc[i, "nconst"], "numVotes")] = dict(stats['numVotes'])
    
###################################################

df = pd.DataFrame.from_dict(dict_df, orient="index").dropna().reset_index()
df = df.rename(columns={'level_0': "nconst", "level_1": "stat"})
df_analysis = df.merge(df1.loc[:, ["nconst", "isActor"]], on="nconst", how="left")
df_analysis.drop(columns=["nconst"]).groupby(by=["stat", "isActor"]).mean()

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


Za nami 0.00%
Za nami 1.52%
Za nami 3.03%
Za nami 4.55%
Za nami 6.06%
Za nami 7.58%
Za nami 9.10%
Za nami 10.61%
Za nami 12.13%
Za nami 13.65%
Za nami 15.16%
Za nami 16.68%
Za nami 18.19%
Za nami 19.71%
Za nami 21.23%
Za nami 22.74%
Za nami 24.26%
Za nami 25.77%
Za nami 27.29%
Za nami 28.81%
Za nami 30.32%
Za nami 31.84%
Za nami 33.36%
Za nami 34.87%
Za nami 36.39%
Za nami 37.90%
Za nami 39.42%
Za nami 40.94%
Za nami 42.45%
Za nami 43.97%
Za nami 45.48%
Za nami 47.00%
Za nami 48.52%
Za nami 50.03%
Za nami 51.55%
Za nami 53.07%
Za nami 54.58%
Za nami 56.10%
Za nami 57.61%
Za nami 59.13%
Za nami 60.65%
Za nami 62.16%
Za nami 63.68%
Za nami 65.19%
Za nami 66.71%
Za nami 68.23%
Za nami 69.74%
Za nami 71.26%
Za nami 72.78%
Za nami 74.29%
Za nami 75.81%
Za nami 77.32%
Za nami 78.84%
Za nami 80.36%
Za nami 81.87%
Za nami 83.39%
Za nami 84.91%
Za nami 86.42%
Za nami 87.94%
Za nami 89.45%
Za nami 90.97%
Za nami 92.49%
Za nami 94.00%
Za nami 95.52%
Za nami 97.03%
Za nami 98.55%


mean            std
stat          isActor                              
averageRating False         6.592772       0.706887
              True          6.588767       0.720223
numVotes      False    178309.448492  137168.809953
              True     169205.555651  140904.142778